In [1]:
import pymysql.cursors
import time
from matplotlib import pyplot as plt
from neo4j import GraphDatabase

In [6]:
connection = pymysql.connect(host='localhost',
                            user='root',
                            password='El3ctron5',
                            database='traffic',
                            charset='utf8',
                            cursorclass=pymysql.cursors.DictCursor)
# Open connection and cursor
with connection:
    with connection.cursor() as cursor:
            start = time.time()
            cursor.execute("select * from segments as seg1 " + \
                           "join segments as seg2 on ((seg1.x1 = seg2.x1 and seg1.y1 = seg2.y1) " + \
                           "or (seg1.x2 = seg2.x1 and seg1.y2 = seg2.y1) " + \
                           "or (seg1.x1 = seg2.x2 and seg1.y1 = seg2.y2) " + \
                           "or (seg1.x2 = seg2.x2 and seg1.y2 = seg2.y2)) " + \
                           "where seg1.street = 'Saint Alphonsus' " + \
                           "and seg2.street = 'Huntington';")
            mid = time.time()
            cursor.execute("select * from segments as seg1 " + \
                           "join segments as seg2 on ((seg1.x1 = seg2.x1 and seg1.y1 = seg2.y1) " + \
                           "or (seg1.x2 = seg2.x1 and seg1.y2 = seg2.y1) " + \
                           "or (seg1.x1 = seg2.x2 and seg1.y1 = seg2.y2) " + \
                           "or (seg1.x2 = seg2.x2 and seg1.y2 = seg2.y2)) " + \
                           "where seg1.street = 'Saint Alphonsus';")
            end = time.time()
            cursor.execute("select * from segments;")
            end1 = time.time()
sql_time1 = mid - start
sql_time2 = end - mid
sql_time3 = end1 - end
print(sql_time1)
print(sql_time2)
print(sql_time3)

0.0938723087310791
0.43639039993286133
1.4288718700408936


In [3]:
# code copied from https://towardsdatascience.com/neo4j-cypher-python-7a919a372be7
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [5]:
conn = Neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="El3ctron5")
start = time.time()
conn.query("match(i1)- [s1: Street {name: 'Huntington'}] - (i2:Intersec) - [s2: Street {name: 'Saint Alphonsus'}]-(i3) return i1, i2, i3 ")
mid = time.time()
conn.query("match(i1)- [s1: Street {name: 'Hemenway'}] - (i2:Intersec) - [s2: Street]-(i3) return i1, i2, i3")
end = time.time()
conn.query("match(m) return m")
end1 = time.time()
neo4j1 = mid - start
neo4j2 = end - mid
neo4j3 = end1 - end
print(neo4j1)
print(neo4j2)
print(neo4j3)

0.12465596199035645
0.09676599502563477
4.152183771133423
